In [67]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

## Task1

빈 칸을 채워주세요!

단계별 output이 github 파일에는 남아있으니 그 output과 동일한 형태인지 확인하면서 진행해주시면 됩니다~

In [68]:
#1. 생성할 문장 데이터

sentence = ("Brick walls are there for a reason and you must not think "
            "that the brick walls aren't there to keep us out, but rather "
            "in this way that the brick walls are there to show us how badly we want things.")

In [69]:
#2. 문자 집합 만들기
world_set = list(set(sentence))

## 문제(1): 각 문자에 정수 인코딩 (공백도 하나의 원소로 포함)
vocab = {j : idx for idx,j in enumerate(world_set)}

In [70]:
print(vocab)

{'w': 0, 'l': 1, 'k': 2, 'p': 3, 'r': 4, 't': 5, 'i': 6, 'a': 7, 'e': 8, 'm': 9, 'g': 10, 'f': 11, 'n': 12, "'": 13, 's': 14, 'b': 15, 'd': 16, 'B': 17, 'u': 18, '.': 19, 'c': 20, ' ': 21, ',': 22, 'h': 23, 'y': 24, 'o': 25}


In [71]:
#3. 문자 집합 크기 확인

vocab_size = len(vocab)
print('문자 집합 크기 : {}'.format(vocab_size))

문자 집합 크기 : 26


In [72]:
#4. 하이퍼 파라미터 설정(자유롭게 수정해보세요!)

hidden_size = vocab_size # 같아야 하는 것 확인!
sequence_length = 10  # 너무 길거나 너무 짧게 잡으면 안됩니다!
learning_rate = 0.01

In [73]:
#5. seqence 길이 단위 자르기

# 데이터 구성을 위한 리스트
x_data = []
y_data = []

## 문제(2): 반복문 내에서의 인덱싱을 사용하여 sequence_length 값 단위로 샘플을 잘라 데이터 만들기, y_str은 x_str은 한 칸씩 쉬프트된 sequnce

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i:i + sequence_length]
  y_str = sentence[i + 1 : i + sequence_length+1]
  print(i, x_str, "->", y_str)

  # x_str과 y_str이 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

0 Brick wall -> rick walls
1 rick walls -> ick walls 
2 ick walls  -> ck walls a
3 ck walls a -> k walls ar
4 k walls ar ->  walls are
5  walls are -> walls are 
6 walls are  -> alls are t
7 alls are t -> lls are th
8 lls are th -> ls are the
9 ls are the -> s are ther
10 s are ther ->  are there
11  are there -> are there 
12 are there  -> re there f
13 re there f -> e there fo
14 e there fo ->  there for
15  there for -> there for 
16 there for  -> here for a
17 here for a -> ere for a 
18 ere for a  -> re for a r
19 re for a r -> e for a re
20 e for a re ->  for a rea
21  for a rea -> for a reas
22 for a reas -> or a reaso
23 or a reaso -> r a reason
24 r a reason ->  a reason 
25  a reason  -> a reason a
26 a reason a ->  reason an
27  reason an -> reason and
28 reason and -> eason and 
29 eason and  -> ason and y
30 ason and y -> son and yo
31 son and yo -> on and you
32 on and you -> n and you 
33 n and you  ->  and you m
34  and you m -> and you mu
35 and you mu -> nd you mus
36

In [74]:
# 출력해서 한 칸씩 쉬프트된 것 확인하기!

print(x_data[0])
print(y_data[0])

[17, 4, 6, 20, 2, 21, 0, 7, 1, 1]
[4, 6, 20, 2, 21, 0, 7, 1, 1, 14]


In [75]:
##6. 입력 시퀀스에 대해 원핫인코딩 수행

## 문제(4) : x_data를 원핫인코딩 > numpy의 eye를 쓸 수 있지 않을까?
x_one_hot = [np.concatenate([np.eye(1,26,i[j]) for j in range(10)]) for i in x_data]

##7. 입력 데이터, 레이블데이터 텐서로 변환

## 문제(5) : x_one_hot과 y_data 텐서로 변환 : 둘 다 같은 형식의 텐서로 변환하면 될까?? (FloatTensor, LongTesor 중 맞는 것은?)
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [76]:
##8. 크기 확인
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([188, 10, 26])
레이블의 크기 : torch.Size([188, 10])


In [77]:
##9.원핫인코딩 결과 샘플 확인하기
print(X[0])

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1.,

In [78]:
##10. 레이블 데이터 샘플 확인하기
print(Y[0])

tensor([ 4,  6, 20,  2, 21,  0,  7,  1,  1, 14])


In [79]:
##11. RNN 모델 구현

##문제(6) : 기본 pytorch 인자 넣기 연습 + forward 채우기
### 조건 : rnn layer 2개 쌓기 + 마지막은 fc layer
### batch_fisrt 설정 필요할까? (유튜브 강의 참고)

class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_size = input_dim, hidden_size = hidden_dim,batch_first = True)
    self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias = True)
  
  def forward(self, x):
    x, hidden = self.rnn(x)
    x, hidden = self.rnn(x)
    x = self.fc(x)
    return x

In [80]:
net = Net(vocab_size, hidden_size, 2)

In [81]:
##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

In [82]:
##14. 출력 크기 점검
outputs = net(X)
print(outputs.shape)

torch.Size([188, 10, 26])


In [83]:
##15. Training 시작

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

    print(predict_str)

ooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooo
ooooooooooooooooooooooooooooooooooooooooooooooooooooooooioooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooo
e e              eo             e                    e     e    e                      e     e            e        e       e        e    e                   eo    e  ee                        e    
                                                                                                                                                                                                     
                                                                                                                                                                                                     
          

In [84]:
predict_str

" ick walls are there tor a reason and you must not think that the brick walls are 't there to seep us out, but rather in this way that the brick walls are there to show us how badly we want thinkt "

결과가 어떤가요?? 마지막 에폭의 문장이 그럴싸한가요?

## Task2

위 sentence는 제가 임의로 생성한 문장들입니다.

마음에 드시는 문구 가져오셔서 문장이 어떻게 생성되는지 확인해보세요! 

영어가 아닌 한국어로 시도해보는 것도 좋겠죠? 

수정이 많이 필요(토큰화 등) 할 수 있으나 한번 시도해보시는 것 권장드립니다 :)

위 베이스라인은 어디든 수정하셔도 좋고 조금 더 자연스러운 문장이 나올 수 있게 다양한 시도를 해보세요!

조건 : 문장 3개 이상, 연결성이 있는 문장을 " " 으로 구분하여 ( )에 넣기

In [85]:
#1. 생성할 문장 데이터
sentence = ("Anyone who stops learning is old, whether at twenty or eighty."
            "Anyone who keeps learning stays young." 
            "The greatest thing in life is to keep your mind young.")

In [86]:
#2. 문자 집합 만들기
world_set = list(set(sentence))

#각 문자에 정수 인코딩 (공백도 하나의 원소로 포함)
vocab = {j : idx for idx,j in enumerate(world_set)}

In [87]:
print(vocab)
vocab_size = len(vocab)
print('문자 집합 크기 : {}'.format(vocab_size))

{'w': 0, 'l': 1, 'k': 2, 'p': 3, 'r': 4, 't': 5, 'i': 6, 'a': 7, 'e': 8, 'm': 9, 'g': 10, 'n': 11, 'f': 12, 's': 13, 'd': 14, 'u': 15, '.': 16, 'T': 17, 'A': 18, ' ': 19, ',': 20, 'h': 21, 'y': 22, 'o': 23}
문자 집합 크기 : 24


In [88]:
# 하이퍼 파라미터 설정
hidden_size = vocab_size
sequence_length = 8
learning_rate = 0.01

In [89]:
#5. seqence 길이 단위 자르기

# 데이터 구성을 위한 리스트
x_data = []
y_data = []


for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i:i + sequence_length]
  y_str = sentence[i + 1 : i + sequence_length+1]
  print(i, x_str, "->", y_str)

  # x_str과 y_str이 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

0 Anyone w -> nyone wh
1 nyone wh -> yone who
2 yone who -> one who 
3 one who  -> ne who s
4 ne who s -> e who st
5 e who st ->  who sto
6  who sto -> who stop
7 who stop -> ho stops
8 ho stops -> o stops 
9 o stops  ->  stops l
10  stops l -> stops le
11 stops le -> tops lea
12 tops lea -> ops lear
13 ops lear -> ps learn
14 ps learn -> s learni
15 s learni ->  learnin
16  learnin -> learning
17 learning -> earning 
18 earning  -> arning i
19 arning i -> rning is
20 rning is -> ning is 
21 ning is  -> ing is o
22 ing is o -> ng is ol
23 ng is ol -> g is old
24 g is old ->  is old,
25  is old, -> is old, 
26 is old,  -> s old, w
27 s old, w ->  old, wh
28  old, wh -> old, whe
29 old, whe -> ld, whet
30 ld, whet -> d, wheth
31 d, wheth -> , whethe
32 , whethe ->  whether
33  whether -> whether 
34 whether  -> hether a
35 hether a -> ether at
36 ether at -> ther at 
37 ther at  -> her at t
38 her at t -> er at tw
39 er at tw -> r at twe
40 r at twe ->  at twen
41  at twen -> at twent
42

In [90]:
##6. 입력 시퀀스에 대해 원핫인코딩 수행
x_one_hot = [np.concatenate([np.eye(1,24,i[j]) for j in range(8)]) for i in x_data]

##7. 입력 데이터, 레이블데이터 텐서로 변환
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [91]:
##8. 크기 확인
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([146, 8, 24])
레이블의 크기 : torch.Size([146, 8])


In [92]:
##9.원핫인코딩 결과 샘플 확인하기
print(X[0])

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.]])


In [93]:
##10. 레이블 데이터 샘플 확인하기
print(Y[0])

tensor([11, 22, 23, 11,  8, 19,  0, 21])


In [94]:
##11. RNN 모델 구현

##문제(6) : 기본 pytorch 인자 넣기 연습 + forward 채우기
### 조건 : rnn layer 2개 쌓기 + 마지막은 fc layer
### batch_fisrt 설정 필요할까? (유튜브 강의 참고)

class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_size = input_dim, hidden_size = hidden_dim,batch_first = True)
    self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias = True)
  
  def forward(self, x):
    x, hidden = self.rnn(x)
    x, hidden = self.rnn(x)
    x = self.fc(x)
    return x

In [95]:
net = Net(vocab_size, hidden_size, 2)

In [96]:
##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

In [97]:
##14. 출력 크기 점검
outputs = net(X)
print(outputs.shape)

torch.Size([146, 8, 24])


In [98]:
##15. Training 시작

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

    print(predict_str)

mmuumuuAmyuAmyuummuuumuummuumAummuAmmummuumummAuuAmuAumuummmuuuuAumuuAmyuAusaummuuumuummAmmuummumummummummuummummmmmummuukmuuuumummmuAusmuAumumuuukumsmum
ssssssusssusssussAsssuussuussssssuAsssssauuusssssssussuusssssssussssusssuAssssuusssuussuussusuyssussussussssssusuassussuusuusssuussssAsssuysssuusssusssus
sssssssssssssssss ssssssssssssssssssssssss ssssssssssssssssssssssssssssssssssss ssssssssss sssssssssssssssssssssssssssssss sssssssssssssssssss ssssssssss
ss ss s s s s s        s   s   s   s ssss      ss  s    sss ssss ss s s s  s         s   s s s sss s s s   ss ss   ss s s   s s     s  s   ss   sss  sss 
s                                                                                                                                                        
                                                                                                                                                         
                                                                            

In [99]:
predict_str

'nyone who stops learning is old, whether at twenty or eighty.Anyone who seeps learning itays young.The greatest thing in life is to keep your mind young.'